In [8]:
import gcsfs
from scipy.io import loadmat
from scipy.signal import resample
from scipy.stats import zscore
import matplotlib.pyplot as plt
import os.path as op
import numpy as np
from neurodsp.filt.filter import filter_signal

In [2]:
fs = gcsfs.GCSFileSystem(project='learning-2-learn-221016')

In [3]:
path = 'learning2learn/Buffalo/wcst-nhp/SA/sess-20180918/ephys/'

In [5]:
data = np.empty((124, 11796177))

for chan in range(1, 5):#125):
    fname = op.join(path, f'chan-{chan}', f'sub-SA_sess-20180918_chan-{chan}.mat')
    print(f"Opening {fname}")
    with fs.open(fname) as f_chan:
        f_chan = loadmat(f_chan)
        # lfp_chan = f_chan['data']
        data[chan-1] = f_chan['data'].squeeze()  #lfp_chan[:]

Opening learning2learn/Buffalo/wcst-nhp/SA/sess-20180918/ephys/chan-1/sub-SA_sess-20180918_chan-1.mat
Opening learning2learn/Buffalo/wcst-nhp/SA/sess-20180918/ephys/chan-2/sub-SA_sess-20180918_chan-2.mat
Opening learning2learn/Buffalo/wcst-nhp/SA/sess-20180918/ephys/chan-3/sub-SA_sess-20180918_chan-3.mat
Opening learning2learn/Buffalo/wcst-nhp/SA/sess-20180918/ephys/chan-4/sub-SA_sess-20180918_chan-4.mat


In [9]:
data.shape

(124, 11796177)

In [ ]:
data = resample(data, data.shape[-1]//10, axis=-1)

In [ ]:
data.shape

In [ ]:
from picard import picard

In [ ]:
K1, W1, Y1 = picard(data[:, :data.shape[-1]//2], n_components=10)
K2, W2, Y2 = picard(data[:, data.shape[-1]//2:], n_components=10)

In [ ]:
w = np.dot(W, K)
mixing = np.dot(w.T, np.linalg.inv(np.dot(w, w.T)))